In [5]:
# Run this cell only once.
# %pip install ultralytics
# !pip install opencv-python-headless
# !pip install labelme2yolo

  Using cached ultralytics-8.1.11-py3-none-any.whl (709 kB)
  Using cached opencv_python-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (62.2 MB)
  Using cached torch-2.2.0-cp311-cp311-manylinux1_x86_64.whl (755.5 MB)
  Using cached torchvision-0.17.0-cp311-cp311-manylinux1_x86_64.whl (6.9 MB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)
  Using cached typing_extensions-4.9.0-py3-none-any.whl (32 kB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cache

In [10]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# GeoTiff Images
import rasterio
from osgeo import gdal

# Visualisation
from matplotlib import pyplot as plt
import matplotlib.image as img
from matplotlib.pyplot import figure
from PIL import Image
import numpy as np

# Model Building
import ultralytics
from ultralytics import YOLO
import labelme2yolo

# Others
import os
import shutil
import zipfile

%matplotlib inline

In [2]:
# 2. Import the data (Only need to run once)

# 2a. Pre-event data
# !wget https://challenge.ey.com/api/v1/storage/admin-files/Pre_Event_San_Juan.tif -O Pre_Event_San_Juan.tif

# 2b.Post-event data
# !wget https://challenge.ey.com/api/v1/storage/admin-files/Post_Event_San_Juan.tif -O Post_Event_San_Juan.tif

In [ ]:
# # 3. Assign the data files to variables 
# pre_event_image = './Pre_Event_San_Juan.tif'
# post_event_image ='./Post_Event_San_Juan.tif'

In [ ]:
# # 4. Data Preprocessing
# # 4a. Create grid size 512x512 for the images (define the function)
# def generate_tiles(input_file, output_dir,grid_x,grid_y):
#     ds = gdal.Open(input_file)

#     # Get image size and number of bands
#     width = ds.RasterXSize
#     height = ds.RasterYSize
#     num_bands = ds.RasterCount

#     # Calculate number of tiles in each dimension
#     num_tiles_x = (width // grid_x)
#     num_tiles_y = (height // grid_y)

#     print(f"Total number of tiles: {num_tiles_x * num_tiles_y}")

#     # Create output directory if it doesn't exist
#     os.makedirs(output_dir, exist_ok=True)

#     # Iterate over each tile and save as a separate TIFF image
#     for i in range(num_tiles_x):
#         for j in range(num_tiles_y):
#             x_offset = i *  grid_x
#             y_offset = j *  grid_y

#             tile_width = min(grid_x, width - x_offset)
#             tile_height = min(grid_y, height - y_offset)

#             tile = []
#             for band in range(1, num_bands + 1):
#                 tile_data = ds.GetRasterBand(band).ReadAsArray(x_offset, y_offset, tile_width, tile_height)
#                 tile.append(tile_data)

#             # Create output filename
#             output_file = os.path.join(output_dir, f"tile_{i}_{j}.tif")
                
#             # Create an output TIFF file with same CRS and band values range
#             driver = gdal.GetDriverByName("GTiff")
#             options = ['COMPRESS=DEFLATE', 'PREDICTOR=2', 'TILED=YES']
#             out_ds = driver.Create(output_file, tile_width, tile_height, num_bands, 
#                        ds.GetRasterBand(1).DataType, options=options)
#             # out_ds = driver.Create(output_file, tile_width, tile_height, num_bands, ds.GetRasterBand(1).DataType)

#             # Set the geotransform
#             geotransform = list(ds.GetGeoTransform())
#             geotransform[0] = geotransform[0] + x_offset * geotransform[1]
#             geotransform[3] = geotransform[3] + y_offset * geotransform[5]
#             out_ds.SetGeoTransform(tuple(geotransform))

#             # Set the projection
#             out_ds.SetProjection(ds.GetProjection())

#             # Write each band to the output file
#             for band in range(1, num_bands + 1):
#                 out_band = out_ds.GetRasterBand(band)
#                 out_band.WriteArray(tile[band - 1])

#             # Close the output file
#             out_ds = None

#     print("Tiles generation completed.")

In [ ]:
# # 4a. Create grid size 512x512 for the images (specify directory for pre event images)
# input_file = "./Pre_Event_San_Juan.tif"
# output_dir = "./Pre_Event_Grids_In_TIFF"
# grid_x = 512
# grid_y = 512
# generate_tiles(input_file, output_dir,grid_x,grid_y)

In [ ]:
# # 4a. Create grid size 512x512 for the images (specify directory for post event images)
# input_file = "./Post_Event_San_Juan.tif"
# output_dir = "./Post_Event_Grids_In_TIFF"
# grid_x = 512
# grid_y = 512
# generate_tiles(input_file, output_dir,grid_x,grid_y)

In [ ]:
# # 4b. Convert TIF images into JPG (define the function)
# def convert_tiff_to_jpeg(input_dir,output_dir):
#     # check if output_dir exists, if not create it
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     for filename in os.listdir(input_dir):
#         # check if file is an image (ends with .tif)
#         if filename.endswith('.tif'):
#             img = Image.open(os.path.join(input_dir, filename))
        
#             # check if image is RGB mode, if not convert it
#             if img.mode != 'RGB':
#                 img = img.convert('RGB')
        
#             # create new filename, replace .tif with .jpg
#             output_filename = os.path.splitext(filename)[0] + '.jpg'
        
#             # save the image in JPEG format
#             img.save(os.path.join(output_dir, output_filename), 'JPEG')
#     print("Conversion from TIFF to JPEG completed.")

In [ ]:
# # 4b. Convert TIF images into JPG (specify directory for pre event images)
# pre_event_input_dir = "./Pre_Event_Grids_In_TIFF"
# pre_event_output_dir = "./Pre_Event_Grids_In_JPEG"
# convert_tiff_to_jpeg(pre_event_input_dir,pre_event_output_dir)

In [ ]:
# # 4b. Convert TIF images into JPG (specify directory for post event images)
# post_event_input_dir = "./Post_Event_Grids_In_TIFF"
# post_event_output_dir = "./Post_Event_Grids_In_JPEG"
# convert_tiff_to_jpeg(post_event_input_dir,post_event_output_dir)

In [ ]:
# # 4c. Rename files in the directory (define the function)
# def rename_files(directory_path,prefix):
# # Define the directory path where your files are located
#     directory_path = directory_path
    
#     # Get a list of all files in the directory
#     files = os.listdir(directory_path)
    
#     # Define a prefix for the new file names 
#     # Change the prefix as per requirement
#     prefix = prefix
    
#     # Start the numbering from 1
#     number = 0
    
#     # Loop through each file in the directory
#     for filename in files:
#         # Check if the item is a file (not a directory)
#         if os.path.isfile(os.path.join(directory_path, filename)):
#             # Get the file extension
#             file_extension = os.path.splitext(filename)[1]
    
#             # Create the new file name with leading zeros
#             new_filename = f"{prefix}{number:03}{file_extension}"
    
#             # Construct the full path to the original and new files
#             old_filepath = os.path.join(directory_path, filename)
#             new_filepath = os.path.join(directory_path, new_filename)
    
#             # Rename the file
#             os.rename(old_filepath, new_filepath)
    
#             # Increment the number for the next file
#             number += 1
    
#     print("Files renamed successfully.")


In [ ]:
# # 4c. Rename files in the directory (pre event images)
# rename_files(pre_event_output_dir, "Pre_Event_")

In [ ]:
# # 4c. Rename files in the directory (post event images)
# rename_files(pre_event_output_dir, "Post_Event_")

In [1]:
# 5. Load the annotated data
import zipfile
import os

# 5a. Create a new folder for the annotated data
os.makedirs('Post_Event_Annotated_JSON', exist_ok=True)
os.makedirs('Pre_Event_Annotated_JSON', exist_ok=True)

def unzip_file(zip_filepath, dest_dir):
    with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
        zip_ref.extractall(dest_dir)

# Usage
unzip_file('./Post_Event_Manual_Annotated.zip', './Post_Event_Annotated_JSON')

In [2]:
# 6. Prepare the file to be YOLO-ready
# 6a. Transform the .json file into .txt format (define the function)
import os, json



In [6]:
# 6b. Generate the configuration file (dataset.yaml)
!labelme2yolo --json_dir ./Post_Event_Annotated_JSON

INFO:labelme2yolo:Converting train set ...
100%|█████████████████████████████████████████████| 9/9 [00:01<00:00,  8.78it/s]
INFO:labelme2yolo:Converting val set ...
100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  8.55it/s]
INFO:labelme2yolo:Converting test set ...
0it [00:00, ?it/s]


In [11]:
# 7. Build the model
#Loading the YOLO model
model = YOLO('yolov8n.pt')
# Display model information (optional)
model.info()

YOLOv8n summary: 225 layers, 3157200 parameters, 0 gradients, 8.9 GFLOPs


(225, 3157200, 0, 8.8575488)

In [27]:
# Train the model on the dataset for 50 epochs
results = model.train(data='dataset.yaml', epochs=50, imgsz=512)

Ultralytics YOLOv8.1.11 🚀 Python-3.11.4 torch-2.2.0+cu121 CPU (Intel Xeon Platinum 8272CL 2.60GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset.yaml, epochs=50, time=None, patience=50, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train63, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_b

RuntimeError: Dataset 'dataset.yaml' error ❌ 
Dataset 'dataset.yaml' images not found ⚠️, missing path '/home/jovyan/PlanetaryComputerExamples/datasets/val'
Note dataset download directory is '/home/jovyan/PlanetaryComputerExamples/datasets'. You can update this in '/home/jovyan/.config/Ultralytics/settings.yaml'

In [24]:
import os

def print_folder_tree(start_path):
    for root, _, files in os.walk(start_path):
        level = root.replace(start_path, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        sub_indent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(sub_indent, f))

# Use the function
print_folder_tree('./val')

val/
    60d2de9ca164a37d24f09b7bbf178de9.png
    60d2de9ca164a37d24f09b7bbf178de9.txt
    6f8eb63b3ca243cd9e224ff89f1f805c.txt
    6f8eb63b3ca243cd9e224ff89f1f805c.png
    .ipynb_checkpoints/
        60d2de9ca164a37d24f09b7bbf178de9-checkpoint.png
        6f8eb63b3ca243cd9e224ff89f1f805c-checkpoint.txt


In [92]:
!yolo task=detect mode=train model= yolov8n.pt data= dataset.yaml

Ultralytics YOLOv8.1.11 🚀 Python-3.11.4 torch-2.2.0+cu121 CPU (Intel Xeon Platinum 8272CL 2.60GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset.yaml, epochs=100, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train50, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

In [89]:
!ls

 CODE_OF_CONDUCT.md				 archive
 CONTRIBUTING.md				 competitions
 ClubPenguin.ipynb				 data.yaml
 LICENSE					 dataset.yaml
'OG Benchmark Notebook.ipynb'			 datasets
'Phase-1 Benchmark Notebook 2024 V0.5 1.ipynb'	 docker-compose.yml
 Post_Event_Annotated_JSON			 labelme
 Post_Event_Grids_In_JPEG			 pyproject.toml
 Post_Event_Grids_In_TIFF			 quickstarts
 Post_Event_Manual_Annotated.zip		 runs
 Pre_Event_Grids_In_JPEG			 scripts
 Pre_Event_Grids_In_TIFF			 train
 Pre_Event_San_Juan.tif				 tutorials
 README.md					 val
 SECURITY.md					 yolov8n.pt
 SUPPORT.md
